# Data Description
The sensors were sampledat the highest rate each device would support, and we segment thisdata into non-overlapping windows of 128 time steps. We includethe data collected from the 31 smartphones in our experiments.

# Data Processing

In [7]:
import numpy as np
import torch 
import os
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from IPython.core.debugger import Tracer
scaler = StandardScaler()
debug_here = Tracer()

<ipython-input-7-ea82c6e7c2a6>:11: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  debug_here = Tracer()


In [4]:
# Data Cleaning 
data_dir = r"D:\My TS Datasets\HAR_Datasets\WISDM_ar\WISDM_ar_v1.1"
data_file = "WISDM_ar_v1.wi.txt"
def data_cleaning(data_dir,data_file):
    file = open(f'{data_dir}/{data_file}', "r")
    data = file.readlines()
    data_subjects = [d.replace(';\n', '').split(',') for d in data]

    # some lines has '' at the end
    for i in range(len(data_subjects)):
        if data_subjects[i][-1] == "":
            data_subjects[i] = data_subjects[i][:-1]
    # some lines has ['\n'] 
    for i in range(len(data_subjects)):
        if data_subjects[i] == ['\n']:
            data_subjects.pop(i)
    # detect rows with more than 6
    for i,j in enumerate(data_subjects):
        if len(j) != 6:
            print(i)
    df = pd.DataFrame(data_subjects, columns=('user', 'activity', 'time', 'x', 'y', 'z'))
    return df

In [12]:
def sliding_window(time_series, width, step, order='F'):
    w = np.hstack([time_series[i:1 + i - width or None:step] for i in range(0, width)])
    result = w.reshape((int(len(w) / width), width), order='F')
    if order == 'F':
        return result
    else:
        return np.ascontiguousarray(result)

def calc_normalization(data):
    data[np.isnan(data)] = 0
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (num_instances, -1))
    scaler.fit(data)
    return scaler
def apply_normalization(data, scaler):
#     scaler = StandardScaler()
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (num_instances, -1))
    norm_data = scaler.transform(data)
    norm_data[np.isnan(norm_data)] = 0
    norm_data = np.reshape(norm_data, (num_instances, num_time_steps, num_features))
    return norm_data

def Wisdom_data_generator(data_dir):
    full_data={}
    seq_length = 128
    shifting_step = 128
    num_variables = 3
    # dataloading 
    # Reading phones accelemeter datasets 
    data_dir = 'WISDM_ar_v1.1'
    wisdom_refined = pd.read_csv(f'{data_dir}/Wisdom_ar.csv')
#     df['activity'] = df['activity'].astype('category')
#     df['labels'] = df['activity'].cat.codes

    # drop irrelevant columns 
#     wisdom_refined = df.drop(columns=['time', 'activity'])
    # drop irrelevant columns 

    # looping trough users 
    for user_name, user_data in  wisdom_refined.groupby('user'):
        data, labels = [], []
        # looping through data from each class per user
        for class_name, class_data in user_data.groupby('labels'):
            col_index=0
            # slicing data to 128 with no overlap 
            sliced_data = np.empty((int(class_data.shape[0]/seq_length),seq_length, num_variables))
            for column in class_data[['x', 'y', 'z']]:
                column_data = class_data[column]
                sliced_data[:,:,col_index] = sliding_window(column_data.values, seq_length,shifting_step, 'T')
                col_index+=1
           # append data 
            data.append(sliced_data)
            # generate labels
            class_labels = np.empty(sliced_data.shape[0])
            class_labels.fill(class_name)
            labels.append(class_labels.astype(int))
            
        # data and labels for each users 
        array_user_data= np.concatenate(data, axis=0 )
        array_user_labels= np.concatenate(labels, axis=0 )        
        
        # Stratified train, validation, test split of the data 
        X_train, X_test, y_train, y_test = train_test_split(array_user_data, array_user_labels,  stratify=array_user_labels,  test_size=0.3,random_state=1)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=1)
        
        # Data normalization 
        # Calculate mean and standard deviation based on train
        scaler = calc_normalization (X_train)
        
        # Apply normalization 
        X_train = apply_normalization(X_train,scaler)
        X_val = apply_normalization(X_val,scaler)
        X_test = apply_normalization(X_test,scaler)
        
        # prepare samples
        train_data = {'samples':X_train, 'labels':y_train}
#         val_data   = {'samples':X_val, 'labels':y_val}
        test_data  = {'samples':X_test, 'labels':y_test}
        os.makedirs('Wisdom_user_data',exist_ok= True)
        torch.save(train_data, f'wisdom_train_test/train_{user_name-1}.pt')
#         torch.save(val_data,  f'Wisdom_user_data/val_{user_name}.pt')
        torch.save(test_data, f'wisdom_train_test/test_{user_name-1}.pt')

In [13]:
dir_data = r"D:\My TS Datasets\HAR_Datasets\WISDM_ar\WISDM_ar_v1.1"
Wisdom_data_generator(dir_data)